In [1]:
!which python

/home/hienpth/miniconda3/envs/p4ds-env/bin/python


# Final project: Phân tích dữ liệu Data Engineer Jobs
Các thành viên:

1. Phạm Thị Hoài Hiền - 18120178

2. Vương Thị Ngọc Linh - 18120195

In [ ]:
import numpy as np 
import matplotlib as plt
import pandas as pd

## Giới thiệu về dữ liệu

Đây là dữ liệu về việc làm Data Engineer (có xen lẫn vài nghề khác). Link: https://www.kaggle.com/andrewmvd/data-engineer-jobs

(chém gió vô đây)

## Tiền xử lí

(chém gió vô đây)

## Các câu hỏi cần trả lời

(chém gió vô đây)